In [33]:
# Import SQLite3 module
import sqlite3

In [34]:
conn2 = sqlite3.connect("goofy.db")

In [35]:
# Remember to add a cursor
cursor2 = conn2.cursor()

In [8]:
# Create a table in the database using "execute" and "CREATE TABLE"
conn2.execute('''CREATE TABLE disney_characters
(id INTEGER PRIMARY KEY,
name TEXT NOT NULL UNIQUE,
city TEXT NOT NULL,
age INTEGER,
birthday datetime,
rating REAL NOT NULL
)
''')

In [14]:
# Add a row of values in the table using "execute" and "INSERT INTO"
# (Commands that alter the table cannot be repeated;
# the script is *creating* and updating a real table outside the Python environment.)
conn2.execute('''
INSERT INTO disney_characters VALUES (102, "Mickey", "Mouseville", 16, "1911-11-04", 3)
''')

In [13]:
# Change values using "UPDATE"
# Print using SELECT and "fetchall"
conn2.execute('''
UPDATE disney_characters
SET birthday = "1921-04-11"
WHERE name = "Goofy";
''')
conn2.execute('''
SELECT * FROM disney_characters;
''').fetchall()

[(101, 'Goofy', 'Mouseville', 76, '1921-04-11', 5.0)]

In [16]:
conn2.execute('''
SELECT * FROM disney_characters;
''').fetchall()

[(101, 'Goofy', 'Mouseville', 76, '1921-04-11', 5.0),
 (102, 'Mickey', 'Mouseville', 16, '1911-11-04', 3.0)]

In [20]:
# Inserting multiple characters using a Python list
rufus = (15, "Rufus", "Pizza", "2021-11-19") # This is a tuple! Tuples can have more than two elements within them.
print(type(rufus))
# Create the list
my_fav_characters = [
    (103, "Donald", "Duckburg", 34, "1978-12-09", 7),
    (104, "Macchia Nera", "Topolinia", 25, "1956-07-07", 10),
    (105, "Clarabella", "Topolinia", 64, "1922-03-05", -14),
    (106, "Bum Bum Ghigno", "Paperopoli", 54, "1903-11-78", 21)
]
# Use "executemany" to read from the list; each question mark in the call represents a column (field)--NOT a row
# Note that now we are instructing the **cursor**, not invoking the **connection** directly
cursor2.executemany('''INSERT INTO disney_characters VALUES (?, ?, ?, ?, ?, ?)''', my_fav_characters)
# It looks like this "cursor" function creates and stores a number of commands to be run through SQL; 
# the "commit" function that follows executes them all.
conn2.commit()

<class 'tuple'>


In [21]:
# Different ways to print rows from the table: .fetchone(), .fetchmany(), fetchall()
cursor2.execute("SELECT * from disney_characters").fetchone() # Retrieves one result

(101, 'Goofy', 'Mouseville', 76, '1921-04-11', 5.0)

In [22]:
cursor2.execute("SELECT * from disney_characters").fetchmany(3) # Retrieves n number of results

[(101, 'Goofy', 'Mouseville', 76, '1921-04-11', 5.0),
 (102, 'Mickey', 'Mouseville', 16, '1911-11-04', 3.0),
 (103, 'Donald', 'Duckburg', 34, '1978-12-09', 7.0)]

In [23]:
cursor2.execute("SELECT * from disney_characters").fetchall() # Retrieves all the results

[(101, 'Goofy', 'Mouseville', 76, '1921-04-11', 5.0),
 (102, 'Mickey', 'Mouseville', 16, '1911-11-04', 3.0),
 (103, 'Donald', 'Duckburg', 34, '1978-12-09', 7.0),
 (104, 'Macchia Nera', 'Topolinia', 25, '1956-07-07', 10.0),
 (105, 'Clarabella', 'Topolinia', 64, '1922-03-05', -14.0),
 (106, 'Bum Bum Ghigno', 'Paperopoli', 54, '1903-11-78', 21.0)]

In [24]:
# We can use more complex SQL syntax within these statements
cursor2.execute("""
            SELECT COUNT(*)
            FROM disney_characters
            WHERE city = "Topolinia";
            """).fetchone()

(2,)

In [29]:
# What about retrieving columns (series) from this table?
disney_ages = cursor2.execute("SELECT age FROM disney_characters;").fetchall()
print(disney_ages)
print(type(disney_ages))
print(type(disney_ages[0]))
# It's a list--a weird list of tuples, each one containing only one element.
average_age = sum([age[0] for age in disney_ages]) / len(disney_ages)
print(average_age)

[(76,), (16,), (34,), (25,), (64,), (54,)]
<class 'list'>
<class 'tuple'>
44.833333333333336


In [32]:
# Finally, something about integration with Pandas; this makes everything easier
import pandas as pd
df = pd.read_sql_query('''SELECT * FROM disney_characters;''', conn2) # Note that I am still using "conn2" as the connection
print(df)
print(df["rating"].mean())

    id            name        city  age    birthday  rating
0  101           Goofy  Mouseville   76  1921-04-11     5.0
1  102          Mickey  Mouseville   16  1911-11-04     3.0
2  103          Donald    Duckburg   34  1978-12-09     7.0
3  104    Macchia Nera   Topolinia   25  1956-07-07    10.0
4  105      Clarabella   Topolinia   64  1922-03-05   -14.0
5  106  Bum Bum Ghigno  Paperopoli   54  1903-11-78    21.0
5.333333333333333


In [41]:
# Let's try some string concatenation
def get_name_city(person_id):
    query = "SELECT name || ' is from ' || city FROM disney_characters WHERE id=" + str(person_id) + ";"
    # Note how we are creating a string that corresponds to a query.
    cursor2.execute(query)
    results = cursor2.fetchall()
    # It looks as if the cursor can store results that are then retrieved as a variable.
    print(results)
    return results[0][0]

print(get_name_city(102))
print(get_name_city(105))

[('Mickey is from Mouseville',)]
Mickey is from Mouseville
[('Clarabella is from Topolinia',)]
Clarabella is from Topolinia


In [45]:
# Let's add a function that adds new rows
def add_character(new_character):
    query = "INSERT INTO disney_characters(id, name, city, age, birthday, rating) VALUES (?, ?, ?, ?, ?, ?);"
    cursor2.execute(query, list(new_character))

petulia = (109, "Petulia", "Topolinia", 53, "1938-12-12", 6)
add_character(petulia)
print(get_name_city(109))

[('Petulia is from Topolinia',)]
Petulia is from Topolinia


In [46]:
# Remember to close the connection at the end of the program.
cursor2.close()
conn2.close()